In [ ]:
import rpy2
import rpy2.robjects.lib.ggplot2 as ggplot2
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
base = importr('base')

# the base of rpy2 plotting is matplotlib, thus we need to declare
# it inline in order to see the plots in the notebook
%matplotlib inline

# we need to activate the automatic conversion for pandas
from rpy2.robjects import pandas2ri
pandas2ri.activate()

# load the needed extension for the %%R cell magic
%load_ext rpy2.ipython

import numpy as np
import pandas as pd

%config IPCompleter.greedy=True

import matplotlib.pylab as plt
plt.rcParams['figure.dpi'] = 200

In [ ]:
import ray
ray.init()

In [ ]:
rootdir = "~/burrito"
outdirs = ["kv-cloudlab-a386584"]
datapaths = ['kernel', 'dpdkthread', 'dpdkmulti', 'shenango_channel']
#num_shards = [2, 4, 6, 8]
num_shards = [4,]
#ops = [1000, 5000, 10000, 20000, 40000, 60000, 80000, 100000, 160000, 200000, 240000, 320000]
ops = [10000, 20000, 40000, 60000, 80000, 100000, 120000, 140000, 160000, 180000, 200000]
#clients = ['10.1.1.3', '10.1.1.4', '10.1.1.5']
clients = ['10.10.1.1']
#shard_types = ['client', 'server', 'basicclient']
shard_types = ['client']
client_concurrency = [4, 8]
poisson = [True]

In [ ]:
# note: this aggregates requestclients together

@ray.remote
def read_exp(outdir, dp, ns, st, o, p, c):
    def get_clients():
        for cl in clients:
            fn = f"{rootdir}/{outdir}/{dp}-{ns}-{st}shard-{o}-poisson={p}-workloadb-{c}-0-client0-{cl}.data"
            try:
                df_file = pd.read_csv(fn, sep=" ")
                df_file["ShardType"] = st
                df_file['NumShards'] = ns
                df_file["Ops"] = o
                df_file = df_file[(df_file.NumOps > 200e3) & (df_file.NumShards > 1)]
                yield df_file
            except Exception as e:
                print(e)
                continue
    
    try:
        exp_df = pd.concat(get_clients())
    except Exception:
        return pd.DataFrame()
        
    # now calculate this experiment's stats.
    exp_stats = exp_df.groupby(["ShardType", "NumShards", "Ops"]).quantile([0.05, 0.25, 0.5, 0.75, 0.95])
    exp_stats = exp_stats.unstack()
    exp_stats.columns = [f"{s}_{t}" for s,t in exp_stats.columns.values]
    exp_stats.columns.to_flat_index()
    exp_stats["Commit"] = outdir
    exp_stats["Datapath"] = dp
    exp_stats["Concurrency"] = c
    exp_stats["Poisson"] = p
    return exp_stats

def combos():
    for outdir in outdirs:
        for dp in datapaths:
            for ns in num_shards:
                for st in shard_types:
                    for o in ops:
                        o = int(o)
                        for p in poisson:
                            for c in client_concurrency:
                                yield read_exp.remote(outdir, dp, ns, st, o, p, c)

exp_dfs = ray.get([x for x in combos()])
df = pd.concat(exp_dfs)
df = df.reset_index()
df

In [ ]:
%%R -i df -w 900 -h 600

library(ggplot2)

#geom_boxplot(aes(group=interaction(ShardType,Ops)), position="dodge") + 
#ggplot(pdf, aes(x=Ops, y=Latency_us, 
#  fill=ShardType, group=interaction(ShardType,Ops))) + 
#    stat_summary(fun.data = f, geom="boxplot", position="dodge") + 
#    facet_grid(NumShards~Concurrency) +
#    coord_cartesian(ylim=c(0, 10000))

ggplot(df, aes(x=Ops, fill=Datapath, group=interaction(Datapath,Ops))) +
  geom_errorbar(aes(ymin=Latency_us_0.05, ymax = Latency_us_0.95), position="dodge") +
  geom_crossbar(aes(ymin=Latency_us_0.25, y=Latency_us_0.5, ymax = Latency_us_0.75), position="dodge") +
  coord_cartesian(ylim=c(0, 350)) + facet_wrap(~Concurrency)
    

In [ ]:
df.to_csv('')

In [ ]:
%%R -i df -w 1200 -h 400

df$kOps <- df$Ops / 1e3
df <- df[df$kOps == 40 | df$kOps == 80 | df$kOps == 120 | df$kOps == 160 | df$kOps == 200,]

df$nego <- grepl("_no", df$Datapath, fixed=TRUE)

#labs <- c(
#    "kernel" = "Linux",
#    "kernel_noneg" = "Linux-N",
#    "dpdk" = "DPDK",
#    "dpdk_noneg" = "DPDK-N",
#    "shenango_channel" = "Shenango (Ad.)",
#    "shenango_channel_noneg" = "Shenango (Ad.)-N",
#    "shenango_rt" = "Shenango (Rt.)",
#    "shenango_rt_noneg" = "Shenango (Rt.)-N",
#    "shenango_rt_nochunnels" = "Shenango-C"
#)
#values = c(
#    "dpdk" = "#1f78b4",
#    "dpdk_noneg" = "#a6cee3",
#    "kernel" = "#e31a1c",
#    "kernel_noneg" = "#fb9a99",
#    "shenango_channel" = "#33a02c",
#    "shenango_channel_noneg" = "#b2df8a",
#    "shenango_rt" = "#006d2c",
#    "shenango_rt_noneg" = "#74c476",
#    "shenango_rt_nochunnels" = "#aaaaaa"
#),
labs <- c(
    "kernel" = "Linux",
    "kernel_noneg" = "Linux",
    "dpdk" = "DPDK",
    "dpdk_noneg" = "DPDK",
    "shenango_channel" = "Shenango-A",
    "shenango_channel_noneg" = "Shenango-A",
    "shenango_rt" = "Shenango-R",
    "shenango_rt_noneg" = "Shenango-R",
    "shenango_rt_nochunnels" = "Baseline"
)

df$lab <- labs[df$Datapath]

ggplot(df, aes(x=as.factor(kOps), label=lab, color=Datapath, group=interaction(Datapath,lab,kOps))) +
geom_errorbar(aes(ymin=Latency_us_0.05, ymax = Latency_us_0.95), position="dodge") +
geom_crossbar(aes(ymin=Latency_us_0.25, y=Latency_us_0.5, ymax = Latency_us_0.75), position="dodge") +
geom_text(aes(y=100), angle=90, position=position_dodge(width=0.9), show.legend=FALSE) +
facet_wrap(~nego) +
scale_color_manual(
    labels = labs,
    values = c(
        "dpdk" = "#1f78b4",
        "dpdk_noneg" = "#1f78b4",
        "kernel" = "#e31a1c",
        "kernel_noneg" = "#e31a1c",
        "shenango_channel" = "#33a02c",
        "shenango_channel_noneg" = "#33a02c",
        "shenango_rt" = "#006d2c",
        "shenango_rt_noneg" = "#006d2c",
        "shenango_rt_nochunnels" = "#aaaaaa"
    ),
) +
coord_cartesian(ylim=c(0, 350)) +
xlab("Offered Load kOp/sec") + ylab("Request Latency (μs)") +
theme_minimal() +
theme(
    legend.position = "top",
    legend.title=element_blank(),
    text = element_text(size=12, family="IBM Plex Sans", face="bold"),
    axis.text.y = element_text(size=12, family="IBM Plex Sans")
)

In [ ]:
df.to_csv('./kv-datapaths-2.csv')

## Actual Batch Sizes

In [ ]:
import re
read_sizes = re.compile(r"client_id=([0-9]+).*p5=([0-9]+) p25=([0-9]+) p50=([0-9]+) p75=([0-9]+) p95=([0-9]+) cnt=([0-9]+)")

def batch_sizes(fn):
    x = []
    with open(fn, 'r') as f:
        for line in f:
            if "send_batch_size" not in line:
                continue
            m = read_sizes.search(line)
            x.append({
                'client_id': m.group(1), 
                'p5': int(m.group(2)),
                'p25': int(m.group(3)),
                'p50': int(m.group(4)),
                'p75': int(m.group(5)),
                'p95': int(m.group(6)),
                'cnt': int(m.group(7))
            })
    return x
        

batch_size_data = None
for outdir in outdirs:
    for ns in num_shards:
        for o in ops:
            o = int(o)
            for st in shard_types:
                for cl in clients:
                    for c in client_concurrency:
                        for b in batching:
                            try:
                                fn = f"{rootdir}/{outdir}/{ns}-{st}shard-{o}-batch={b}-wrkloadbunf1-{c}-0-client0-{cl}.out"
                                data = batch_sizes(fn)
                                df_file = pd.DataFrame.from_records(data)
                                df_file["Ops"] = o
                                df_file["Client"] = cl
                                df_file["Concurrency"] = c
                                df_file["Commit"] = outdir
                                df_file["Batching"] = b
                                if batch_size_data is None:
                                    batch_size_data = df_file
                                else:
                                    batch_size_data = batch_size_data.append(df_file)
                            except:
                                print(fn)

batch_size_data
#dfq = df.groupby(["Commit", "ShardType", "NumShards", "Wrkload", "Concurrency", "Batching", "Ops"]).quantile([0.05, 0.25, 0.5, 0.75, 0.95])
#dfq.head()

In [ ]:
%%R -i batch_size_data -w 900 -h 600

ggplot(batch_size_data, aes(x=Ops, fill=Client, group=interaction(Client,Ops))) +
  geom_errorbar(aes(ymin=p5, ymax = p95), position="dodge") +
  geom_crossbar(aes(ymin=p25, y=p50, ymax = p75), position="dodge") +
  facet_grid(Commit~Batching)
#  coord_cartesian(ylim=c(0, 500))

#### With a dedicated load balancer

In [ ]:
lboutdirs = ["kvlb-1883ee8", "kvlb-79b6c33"]
# num_shards = 2 machines, fixed
lbops = [5000, 10000, 20000, 40000, 60000, 80000, 100000]
lbshard_types = ['client', 'server']
lbclients = ['10.1.1.7', '10.1.1.8', '10.1.1.9']
lbclient_concurrency = [4, 8]

In [ ]:
lbdf = None
for lboutdir in lboutdirs:
    for o in lbops:
        for st in lbshard_types:
            for cl in lbclients:
                for c in lbclient_concurrency:
                    fn = f"{lboutdir}/{st}shard-{o}-wrkloadbunf1-{c}-client0-{cl}.data"
                    try:
                        df_file = md.read_csv(fn, sep=" ")
                        df_file = df_file[(df_file.NumOps > 200000)]
                        df_file["Commit"] = lboutdir
                        df_file["Concurrency"] = c
                        if lbdf is None:
                            lbdf = df_file
                        else:
                            lbdf = lbdf.append(df_file)
                    except:
                        continue

lbdfq = lbdf.groupby(["Commit", "ShardType", "NumShards", "Ops", "Concurrency"]).quantile([0.05, 0.25, 0.5, 0.75, 0.95])
lbdfq

In [ ]:
lbdfqu = lbdfq.unstack()
lbdfqu.columns = [f"{s}_{t}" for s,t in lbdfqu.columns.values]
lbdfqu.columns.to_flat_index()
lbdfqu.to_csv('/tmp/kvlb-tmp.csv')
lbdfqu

In [ ]:
plbdf = pd.read_csv('/tmp/kvlb-tmp.csv')

In [ ]:
%%R -i plbdf -w 900 -h 600

#f <- function(x) {
#  r <- quantile(x, probs = c(0.05, 0.25, 0.5, 0.75, 0.95))
#  names(r) <- c("ymin", "lower", "middle", "upper", "ymax")
#  r
#}

ggplot(plbdf, aes(x=Ops, fill=ShardType, group=interaction(ShardType,Ops))) +
  geom_errorbar(aes(ymin=Latency_us_0.05, ymax = Latency_us_0.95), position="dodge") +
  geom_crossbar(aes(ymin=Latency_us_0.25, y=Latency_us_0.5, ymax = Latency_us_0.75), position="dodge") +
  facet_grid(Commit~Concurrency) +
  coord_cartesian(ylim=c(0, 1000))